In [13]:
import pandas as pd

# Load your data
df = pd.read_csv("final_fx_dataset.csv")

# Ensure date column is datetime
df['scrape_date'] = pd.to_datetime(df['scrape_date'])

# Define the common start date
start_date = pd.Timestamp("2025-10-08")

# Drop duplicates within each pair_id + date (keeping the last row)
df = df.drop_duplicates(subset=['pair_id', 'scrape_date'], keep='last')

# Create a filled DataFrame
filled_df = (
    df.groupby('pair_id', group_keys=False)
      .apply(lambda g: (
          g.set_index('scrape_date')
           .sort_index()
           .reindex(pd.date_range(start=max(start_date, g['scrape_date'].min()),
                                  end=g['scrape_date'].max(), freq='D'))
           .ffill().bfill()
           .assign(pair_id=g['pair_id'].iloc[0])
           .reset_index()
           .rename(columns={'index': 'scrape_date'})
      ))
)
# Calculate peak-relative change from base
def peak_relative_change(prices):
    """
    prices: pd.Series of exchange rates for a single pair_id, sorted by date
    Returns: pd.Series of changes relative to peak/base
    """
    base = prices.iloc[0]
    peak = base
    changes = []
    
    for price in prices:
        if price >= peak:
            # new peak: positive change from base or previous peak
            peak = price
            change = (price - base)/ base
        else:
            # if below base, negative change
            change = (price - base)/ base if price < base else (price - base)/base
        changes.append(change)
    return pd.Series(changes, index=prices.index)

filled_df['peak_change'] = filled_df.groupby('pair_id')['avg_price'].transform(peak_relative_change)

filled_df['weighted_spread'] = filled_df['spread'] / filled_df['avg_price']

# Save result
filled_df.to_csv("final_final.csv", index=False)


C:\Users\hp\AppData\Local\Temp\ipykernel_1320\1507263237.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (
